In [17]:
from sklearn import svm
import numpy as np
import cvxopt

In [28]:
# set data
# X = np.array([
#     [1, 0],
#     [0, 1],
#     [0, -1],
#     [-1, 0],
#     [0, 2],
#     [0, -2],
#     [-2, 0]
# ])

# X = np.array([
#     [-2, 0],
#     [4, -3],
#     [4, 1],
#     [6, 0],
#     [10, -5],
#     [10, 3],
#     [10, 3]
# ])

X = np.array([
    [1,2,0,2,0],
    [1,0,2,0,2],
    [1,0,-2,0,2],
    [1,-2,0,2,0],
    [1,0,4,0,8],
    [1,0,-4,0,8],
    [1,-4,0,8,0]
])


Y = np.array([-1, -1, -1, 1, 1, 1, 1])

def kernel(x1, x2):
    tmp = 1 + np.dot(x1, x2)
    return tmp ** 2

In [29]:
# calculate Q

print(X.shape)
ndata, nfeature = X.shape
Q_matrix = np.zeros((ndata, ndata))

# for i in range(ndata):
#     for j in range(i, ndata):
#         Q_matrix[i, j] = Y[i] * Y[j] * kernel(X[i], X[j])
#         Q_matrix[j, i] = Q_matrix[i, j]

for i in range(ndata):
    for j in range(i, ndata):
        Q_matrix[i, j] = Y[i] * Y[j] * np.dot(X[i], X[j])
        Q_matrix[j, i] = Q_matrix[i, j]
    
# for objective function: 1/2x'Qx - 1x
P = cvxopt.matrix(Q_matrix, tc='d')
q = cvxopt.matrix(-np.ones(ndata), tc='d')
# for constraint: Gx <= h (a <= 0)
G = cvxopt.matrix(-np.eye(ndata), tc='d')
h = cvxopt.matrix(np.zeros(ndata), tc='d')
# for constrant: Ax = b (y_n* alpha_n = 0)
A = cvxopt.matrix(Y.reshape(1,-1), tc='d')
b = cvxopt.matrix(np.zeros(1), tc='d')

sol = cvxopt.solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

print([np.round(x, 5) for x in alphas])

(7, 5)
     pcost       dcost       gap    pres   dres
 0: -1.9307e+00 -3.9963e+00  2e+01  3e+00  2e+00
 1: -2.4557e+00 -3.0153e+00  6e+00  1e+00  6e-01
 2: -1.8544e-01 -1.2126e+00  1e+00  4e-16  1e-14
 3: -4.6854e-01 -5.4840e-01  8e-02  2e-16  2e-15
 4: -4.9836e-01 -5.0061e-01  2e-03  7e-17  2e-15
 5: -4.9998e-01 -5.0001e-01  2e-05  7e-17  9e-16
 6: -5.0000e-01 -5.0000e-01  2e-07  7e-17  1e-15
Optimal solution found.
[array([ 0.]), array([ 0.25]), array([ 0.25]), array([ 0.33333]), array([ 0.08333]), array([ 0.08333]), array([ 0.])]


In [30]:
b, w = sol['x'][0], (sol['x'][1:]).T
print(round(b,10))
print([round(x,5) for x in w])


2.389e-07
[0.25, 0.25, 0.33333, 0.08333, 0.08333, 0.0]


In [32]:
import numpy as np
from scipy import optimize

X = np.array([
    [1, 0],
    [0, 1],
    [0, -1],
    [-1, 0],
    [0, 2],
    [0, -2],
    [-2, 0]
])

y = np.array([-1, -1, -1, 1, 1, 1, 1])

def kernel(x, y):
    return(1 + np.dot(x.T, y))**2


# Q
num, dim = X.shape
Q = np.zeros((num, num))
for n in range(num):
    for m in range(num):
        Q[n][m] = y[n]*y[m]*kernel(X[n], X[m])
c = np.ones((num, 1))*-1

alpha = np.ones((num, 1))


def objective(x):
    return (0.5*np.dot(x.T, np.dot(Q, x))+np.dot(c.T, x))


cons = []
for i in range(num):
    def f(x, i=i):
        return x[i]
    cons.append({"type": "ineq", "fun": f})
cons.append({"type": "eq", "fun": lambda x: np.dot(y.T, x)})

res_cons = optimize.minimize(objective, alpha,
                             constraints = cons, method='SLSQP',
                             options={'disp': False})
print(res_cons.x)
print(dir(res_cons))

[ -7.61765893e-13   7.03703695e-01   7.03703695e-01   8.88888879e-01
   2.59259254e-01   2.59259257e-01  -7.40580335e-13]
['fun', 'jac', 'message', 'nfev', 'nit', 'njev', 'status', 'success', 'x']


In [3]:
from sklearn import svm
import numpy as np

# X = np.array([
#     [1, 0],
#     [0, 1],
#     [0, -1],
#     [-1, 0],
#     [0, 2],
#     [0, -2],
#     [-2, 0]
# ])

# X = np.array([
#     [-2, 0],
#     [4, -3],
#     [4, 1],
#     [6, 0],
#     [10, -5],
#     [10, 3],
#     [10, 3]
# ])

X = np.array([
    [1,2,0,2,0],
    [1,0,2,0,2],
    [1,0,-2,0,2],
    [1,-2,0,2,0],
    [1,0,4,0,8],
    [1,0,-4,0,8],
    [1,-4,0,8,0]
])

Y = np.array([-1, -1, -1, 1, 1, 1, 1])
out = svm.SVC(kernel='linear', C = 100000000.0)
out2= svm.LinearSVC(penalty= "l2",C=100000000.0)

out.fit(X, Y)
out2.fit(X, Y)

print([np.round(i, 3) for i in out.coef_])
print(out.coef0)

print([np.round(i, 3) for i in out2.coef_])
print(out2.intercept_)

[array([-0.   , -0.666, -0.   ,  0.666,  0.333])]
0.0
[array([-0.833, -0.667, -0.   ,  0.667,  0.333])]
[-0.83331882]


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import qps

ImportError: No module named 'qps'

In [ ]:
X=[[1,0],[0,1],[0,-1],[-1,0],[0,2],[0,-2],[-2,0]]
X = [[float(i) for i in row] for row in X]
y = [-1,-1,-1,1,1,1,1]

X = np.array(X)
y = np.array(y)

In [ ]:
def kernel(x,y):
    return(1+2*np.dot(x.T,y))**2 
qps.kernelSVMwithQP(X,y,kernel)